# Text Classification of StackOverflow using BiLSTM RNNs with deep self-attention

In [1]:
import sys
import os
from os import pardir, getcwd
from os.path import join, abspath
PARENT_DIRECTORY = abspath(join(getcwd(), pardir))
sys.path.insert(0, PARENT_DIRECTORY)

import warnings
import sklearn.exceptions
import talos as ta
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

from definitions import  TALOS_DIR
from app.preprocessing import (load_dataset, load_embeddings,
                               preprocess_data, save_embeddings_matrix)
from app.models import load_bi_lstm_model, find_best_model_over_scan_logs
from app.metrics import *

#Comment out In case of Testing use only a set of the tags as dataset
# tags_categories = ['c', 'python', 'java']
# RUN_STATE = 'testing'

#Comment out In case of Production use all the tags of the dataset
tags_categories = "__all__"
RUN_STATE = 'production'

Using TensorFlow backend.


## Preprocessing for the loaded Dataset
1. Format into *lowercase*
2. Remove some of the *punctuation* characters
3. Remove *Numbers*
4. Remove *stopwords*
5. Remove *links*

In [2]:
data = load_dataset(tags_categories=tags_categories, load_from_pickle=True)
classes_counts =data['tags'].value_counts().where(lambda cls: cls > 0).dropna() 
Classes = list(classes_counts.index)
Nclasses = len(Classes)
print(classes_counts, Classes, Nclasses)

sql              2000
ruby-on-rails    2000
android          2000
angularjs        2000
asp.net          2000
c                2000
c#               2000
c++              2000
css              2000
html             2000
ios              2000
iphone           2000
java             2000
javascript       2000
jquery           2000
mysql            2000
objective-c      2000
php              2000
python           2000
.net             2000
Name: tags, dtype: int64 ['sql', 'ruby-on-rails', 'android', 'angularjs', 'asp.net', 'c', 'c#', 'c++', 'css', 'html', 'ios', 'iphone', 'java', 'javascript', 'jquery', 'mysql', 'objective-c', 'php', 'python', '.net'] 20


 ### Hyper parameter tuning for the  model

In [3]:
# 70% Train & 30% Test
# 70% Train-Dev % 30* Train-Dev 
embeddings_voc, embeddings_vec = load_embeddings(load_from_pickle=True)
model_data = preprocess_data(data, 'tags', 'post', cv_split_dev=0.125)
embeddings_matrix_path = save_embeddings_matrix(embeddings_voc, embeddings_vec, model_data['words_index'])


In [4]:
model_data['x_train'][4]

array([  54,   19, 2288,  766,  505,  211,  766, 1806,  766, 1923,   69,
         81,  666,  149,   39,  202,  505,   94,   19,   54, 2288,  766,
        505, 1335,  835, 2829,  766, 1560,  943,  513,  149,  310,  526,
        156,  572, 2535,   65,   96, 1442,  101,  853,    2,   65,  101,
         34,  144,  839,    2,   34,  224, 1166,  202,  505,   19,   54,
       2288, 2829,  766,  943,  513,  149,  835,  526,  105,  278,  545,
        572,   54,   19, 2288,  513,  766, 1923,  108,  835,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

### UniStacked RNN with BiLSTM & MLP on top of it

In [5]:
###### Production configuration
rnn_deep_lstm_config = {
    "model_type": ["keras_deep_LSTM_model"],
    "embedding_dim": [embeddings_vec.shape[1]],
    "lstm_size": [200],
    "dense": [300],
    "embeddings_matrix_path": [embeddings_matrix_path],
    "visualize_process": [True],
    "with_early_stoping": [True],
    "multistack_run": [False],
    'early_stopping':[True],
    'early_stopping_config__monitor': ['val_f1'],
    'early_stopping_config__min_delta': [0],
    'early_stopping_config__patience': [5],
    'early_stopping_config__mode': ['max'],
    "embeddings_dropout": [0.2],
    "var_dropout": [0.2, 0.6],
    "mlp_dropout": [0.2],
    "mlp_activation": ["softmax"],
    "rnn_activation": ["relu", "tanh"],
    "optimizer": ["Nadam", "Adam"],
    "batch_size": [32, 64],
    "epochs": [3 if RUN_STATE == 'testing' else 10]
}

In [6]:
TALOS_LSTM_DEEP_LOG_FILENAME = 'talos_lstm_deep_log'
talos_lstm_deep_log_pathname = os.path.join(TALOS_DIR, TALOS_LSTM_DEEP_LOG_FILENAME)

history_model = ta.Scan(model_data['x_train'],
                        model_data['y_train'],
                        x_val=model_data['x_train_dev'],
                        y_val=model_data['y_train_dev'],
                        model=load_bi_lstm_model,
                        params=rnn_deep_lstm_config,
                        grid_downsample=0.1,
                        print_params=True,
                        last_epoch_value=True,
                        seed=(123),
                        dataset_name=talos_lstm_deep_log_pathname
                        )


  0%|          | 0/1 [00:00<?, ?it/s]WARNING: Logging before flag parsing goes to stderr.
W0715 03:55:03.272535 140549419595584 deprecation_wrapper.py:119] From /home/giannhs/miniconda3/envs/text_analytics/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0715 03:55:03.288831 140549419595584 deprecation_wrapper.py:119] From /home/giannhs/miniconda3/envs/text_analytics/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0715 03:55:03.343727 140549419595584 deprecation_wrapper.py:119] From /home/giannhs/miniconda3/envs/text_analytics/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0715 03:55:03.354533 140549419595584 deprecation_wrapper.py:119] From /home/giannhs/minicon

{'model_type': 'keras_deep_LSTM_model', 'embedding_dim': 300, 'lstm_size': 200, 'dense': 300, 'embeddings_matrix_path': 'embeddings-matrix-pickle', 'visualize_process': True, 'with_early_stoping': True, 'multistack_run': False, 'early_stopping': True, 'early_stopping_config__monitor': 'val_f1', 'early_stopping_config__min_delta': 0, 'early_stopping_config__patience': 5, 'early_stopping_config__mode': 'max', 'embeddings_dropout': 0.2, 'var_dropout': 0.2, 'mlp_dropout': 0.2, 'mlp_activation': 'softmax', 'rnn_activation': 'tanh', 'optimizer': 'Adam', 'batch_size': 64, 'epochs': 10}


W0715 03:55:03.477378 140549419595584 deprecation.py:506] From /home/giannhs/miniconda3/envs/text_analytics/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0715 03:55:03.803448 140549419595584 deprecation.py:323] From /home/giannhs/miniconda3/envs/text_analytics/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2974: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0715 03:55:04.168659 140549419595584 deprecation_wrapper.py:119] From /home/giannhs/miniconda3/envs/text_analytics/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 600)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 600, 300)          6000600   
_________________________________________________________________
dropout_1 (Dropout)          (None, 600, 300)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 600, 400)          801600    
_________________________________________________________________
deep_attention_1 (DeepAttent [(None, 400), (None, 600) 160801    
_________________________________________________________________
dense_1 (Dense)              (None, 300)               120300    
_________________________________________________________________
dense_2 (Dense)              (None, 20)                6020      
Total para


Epoch 00001: val_accuracy improved from -inf to 0.97056, saving model to keras_deep_LSTM_model

Epoch 00001: val_f1 improved from -inf to 0.61567, saving model to keras_deep_LSTM_model


...


Epoch 00007: val_accuracy did not improve from 0.98221

Epoch 00007: val_f1 did not improve from 0.80572



Epoch 00008: val_accuracy improved from 0.98221 to 0.98254, saving model to keras_deep_LSTM_model

Epoch 00008: val_f1 improved from 0.80572 to 0.81055, saving model to keras_deep_LSTM_model



Epoch 00009: val_accuracy did not improve from 0.98254

Epoch 00009: val_f1 improved from 0.81055 to 0.81231, saving model to keras_deep_LSTM_model



Epoch 00010: val_accuracy improved from 0.98254 to 0.98261, saving model to keras_deep_LSTM_model

Epoch 00010: val_f1 improved from 0.81231 to 0.81415, saving model to keras_deep_LSTM_model


100%|██████████| 1/1 [3:39:52<00:00, 13192.96s/it]


    Finds the best model configuration set for the LSTM with deep self-attention, after the Talos Scanning.

In [7]:
report_talos = ta.Reporting(history_model)
best_model_idx = report_talos.data['val_f1'].idxmax()
best_model_params = report_talos.data.loc[best_model_idx].to_dict()
best_model_params

{'round_epochs': 10,
 'val_loss': 0.6147428828307561,
 'val_precision': 0.872153952053615,
 'val_recall': 0.7642857144900731,
 'val_f1': 0.8141458122389658,
 'val_accuracy': 0.9826142841066633,
 'val_categorical_accuracy': 0.8108571427209037,
 'loss': 0.48062803679096455,
 'precision': 0.893560794927636,
 'recall': 0.7891836734401937,
 'f1': 0.8376876486272228,
 'accuracy': 0.984759175845555,
 'categorical_accuracy': 0.8384897959086359,
 'model_type': 'keras_deep_LSTM_model',
 'embedding_dim': 300,
 'lstm_size': 200,
 'dense': 300,
 'embeddings_matrix_path': 'embeddings-matrix-pickle',
 'visualize_process': 'True',
 'with_early_stoping': 'True',
 'multistack_run': 'False',
 'early_stopping': 'True',
 'early_stopping_config__monitor': 'val_f1',
 'early_stopping_config__min_delta': 0,
 'early_stopping_config__patience': 5,
 'early_stopping_config__mode': 'max',
 'embeddings_dropout': 0.2,
 'var_dropout': 0.2,
 'mlp_dropout': 0.2,
 'mlp_activation': 'softmax',
 'rnn_activation': 'tanh',
 

    Train return am RNN LSTM model with the the best configuration set.

In [8]:
if RUN_STATE == 'testing':
    # Train and Load the best model of given the tuned featured model
    best_model_params['early_stopping'] = True
    best_model_params['with_early_stopping'] = True
    best_model_params['visualize_process'] = True
    model_history, model = load_bi_lstm_model(model_data['x_train'],
                                              model_data['y_train'],
                                              model_data['x_train_dev'],
                                              model_data['y_train_dev'],
                                              best_model_params)

### Visualize Model History Scores

In [9]:
if RUN_STATE == 'testing':
    from app.visualization import plot_history_metrics
    import matplotlib.pylab as plt

    %matplotlib inline
    plot_history_metrics(history_obj=model_history)

### Evaluate performance model

Evaluates the performance of the best trained model in the **test** dataset. 

In [10]:
if RUN_STATE == 'testing':
    score = model.evaluate(model_data['x_test'],
                           model_data['y_test'],
                           batch_size=best_model_params['batch_size'],
                           verbose=1)

    print('\nTest f1: %.4f' % (score[1]))
    print('\nTest categorical accuracy: %.4f'% (score[2]))

### Visualize Prediction Perfomance  model

In [11]:
if RUN_STATE == 'testing':
    import numpy as np
    from app.visualization import (plot_prediction_metrics,
                                   create_clf_report,
                                   plot_roc_curve,
                                   plot_precision_recall_curve,
                                   plot_confusion_matrix)
    import matplotlib.pylab as plt

    prediction_val = model.predict(model_data['x_test'], batch_size=best_model_params['batch_size'])

    # returns each entry result to the classification with the relevant probabilities
    y_pred_processed = np.array([np.argmax(val) for val in prediction_val])
    y_true_processed = np.array([np.argmax(val) for val in model_data['y_test']])

    # If you want to see the OneVSAll ROC Curves of each class uncomment the below line
    # plot_roc_curve(model_data['y_test'], prediction_val, Classes, 1)

    # If you want to see the OneVSAll Precission Recall Curves of each class, comment out the below line
    # plot_precision_recall_curve(model_data['y_test'], prediction_val, Classes , 1)

    # If you want to get the Classification Report, comment out the below line
    create_clf_report(model_data['y_test'], (prediction_val > 0.5).astype('int32'),
                      y_true_processed, y_pred_processed)

    # If you want to get the confusion matrix,comment out the below line
    plot_confusion_matrix(y_true_processed, y_pred_processed, Classes)

### Multistack RNN with BiLSTM & MLP on top of it

In [12]:
TALOS_LSTM_DEEP_MULTI_LOG_FILENAME = 'talos_lstm_deep_multi_log'
talos_lstm_deep_multi_log_pathname = os.path.join(TALOS_DIR, TALOS_LSTM_DEEP_MULTI_LOG_FILENAME)


###### Production configuration
rnn_deep_lstm_multi_config = rnn_deep_lstm_config.copy()
rnn_deep_lstm_multi_config.update({
    "model_type": ["keras_deep_LSTM_multi_model"],
    "multistack_run": [True],
})

history_model_multi = ta.Scan(model_data['x_train'],
                        model_data['y_train'],
                        x_val=model_data['x_train_dev'],
                        y_val=model_data['y_train_dev'],
                        model=load_bi_lstm_model,
                        params=rnn_deep_lstm_multi_config,
                        grid_downsample=0.1,
                        print_params=True,
                        last_epoch_value=True,
                        seed=(123),
                        dataset_name=talos_lstm_deep_multi_log_pathname
                        )


  0%|          | 0/1 [00:00<?, ?it/s]

{'model_type': 'keras_deep_LSTM_multi_model', 'embedding_dim': 300, 'lstm_size': 200, 'dense': 300, 'embeddings_matrix_path': 'embeddings-matrix-pickle', 'visualize_process': True, 'with_early_stoping': True, 'multistack_run': True, 'early_stopping': True, 'early_stopping_config__monitor': 'val_f1', 'early_stopping_config__min_delta': 0, 'early_stopping_config__patience': 5, 'early_stopping_config__mode': 'max', 'embeddings_dropout': 0.2, 'var_dropout': 0.6, 'mlp_dropout': 0.2, 'mlp_activation': 'softmax', 'rnn_activation': 'tanh', 'optimizer': 'Nadam', 'batch_size': 64, 'epochs': 10}


W0715 07:34:56.677434 140549419595584 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0715 07:34:56.693991 140549419595584 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0715 07:34:56.713730 140549419595584 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0715 07:34:56.731040 140549419595584 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0715 07:34:56.886838 140549419595584 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 600)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 600, 300)          6000600   
_________________________________________________________________
dropout_1 (Dropout)          (None, 600, 300)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 600, 400)          801600    
_________________________________________________________________
lstm_2 (LSTM)                (None, 600, 200)          480800    
_________________________________________________________________
deep_attention_1 (DeepAttent [(None, 200), (None, 600) 40401     
_________________________________________________________________
dense_1 (Dense)              (None, 300)               60300     
__________


Epoch 00001: val_accuracy improved from -inf to 0.97254, saving model to keras_deep_LSTM_multi_model

Epoch 00001: val_f1 improved from -inf to 0.67709, saving model to keras_deep_LSTM_multi_model


KeyboardInterrupt: 

    Finds the best model configuration set for the Multi LSTM with deep self-attention, after the Talos Scanning.

In [ ]:
report_talos_multi = ta.Reporting(history_model_multi)
best_model_idx = report_talos_multi.data['val_f1'].idxmax()
best_model_params_multi = report_talos_multi.data.loc[best_model_idx].to_dict()
best_model_params_multi

    Train return a Multistack RNN LSTM model with the the best configuration set.

In [ ]:
if RUN_STATE == 'testing':
    # Train and Load the best model of given the tuned featured model
    model_history_multi, model_multi = load_bi_lstm_model(model_data['x_train'],
                                                          model_data['y_train'],
                                                          model_data['x_train_dev'],
                                                          model_data['y_train_dev'],
                                                          best_model_params)

In [ ]:
### Visualize Model History Scores

In [ ]:
if RUN_STATE == 'testing':
    from app.visualization import plot_history_metrics
    import matplotlib.pylab as plt

    %matplotlib inline
    plot_history_metrics(history_obj=model_history_multi)

### Evaluate performance model

Evaluates the performance of the best trained model in the **test** dataset. 

In [ ]:
if RUN_STATE == 'testing':
    score_multi = model_multi.evaluate(model_data['x_test'],
                                       model_data['y_test'],
                                       batch_size=best_model_params_multi['batch_size'],
                                       verbose=1)

    print('\nTest f1: %.4f' % (score_multi[1]))
    print('\nTest categorical accuracy: %.4f'% (score_multi[2]))

In [ ]:
### Visualize Prediction Perfomance  model

In [ ]:
if RUN_STATE == 'testing':
    import numpy as np
    from app.visualization import (plot_prediction_metrics,
                                   create_clf_report,
                                   plot_roc_curve,
                                   plot_precision_recall_curve,
                                   plot_confusion_matrix)
    import matplotlib.pylab as plt

    prediction_val_multi = model.predict(model_data['x_test'], batch_size=best_model_params['batch_size'])

    # returns each entry result to the classification with the relevant probabilities
    y_pred_processed_multi = np.array([np.argmax(val) for val in prediction_val_multi])
    y_true_processed_multi = np.array([np.argmax(val) for val in model_data['y_test']])

    # If you want to see the OneVSAll ROC Curves of each class uncomment the below line
    # plot_roc_curve(model_data['y_test'], prediction_val, Classes, 1)

    # If you want to see the OneVSAll Precission Recall Curves of each class, comment out the below line
    # plot_precision_recall_curve(model_data['y_test'], prediction_val, Classes , 1)

    # If you want to get the Classification Report, comment out the below line
    create_clf_report(model_data['y_test'], (prediction_val_multi > 0.5).astype('int32'),
                      y_true_processed_multi, y_pred_processed_multi)

    # If you want to get the confusion matrix,comment out the below line
    plot_confusion_matrix(y_true_processed_multi, y_pred_processed_multi, Classes)